In [1]:
import requests

import tiktoken
from dotenv import load_dotenv
from tqdm import tqdm
from elasticsearch import Elasticsearch
from openai import OpenAI


In [2]:
load_dotenv("../.env")

True

In [3]:
!curl -s localhost:9200 | findstr build_hash

    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",


In [4]:
docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)


In [5]:
es_client = Elasticsearch("http://localhost:9200")
llm_client = OpenAI()

In [6]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

es_client.count(index=index_name)

100%|██████████| 948/948 [00:59<00:00, 15.89it/s]


ObjectApiResponse({'count': 948, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [8]:
def elastic_search(query, size=5, course=None):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields",
                    }
                },
            }
        },
    }

    if course is not None:
        search_query["query"]["bool"]["filter"] = {"term": {"course": course}}

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return response, result_docs

In [9]:
query = "How do execute a command on a Kubernetes pod?"
response, _ = elastic_search(query)

response["hits"]["hits"][0]["_score"]

44.50556

In [10]:
query = "How do copy a file to a Docker container?"
response, search_results = elastic_search(
    query, size=3, course="machine-learning-zoomcamp"
)

search_results[2]["question"]

'How do I copy files from a different folder into docker container’s working directory?'

In [11]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""

    context_template = """
Q: {question}
A: {text}
""".strip()

    for doc in search_results:
        context = (
            context
            + context_template.format(question=doc["question"], text=doc["text"])
            + "\n\n"
        )

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [12]:
prompt = build_prompt(query, search_results)

len(prompt)

1447

In [13]:
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)

len(tokens)

321

In [14]:
def llm(prompt):
    response = llm_client.chat.completions.create(
        model="gpt-4o", messages=[{"role": "user", "content": prompt}]
    )

    prompt_tokens = response.usage.prompt_tokens
    completion_tokens = response.usage.completion_tokens
    total_tokens = response.usage.total_tokens
    
    return (
        response.choices[0].message.content,
        {
            "prompt_tokens": prompt_tokens,
            "completion_tokens": completion_tokens,
            "total_tokens": total_tokens,
        }
    )

In [15]:
llm_response, llm_tokens = llm(prompt)

print(llm_response)

To copy a file from your local machine to a Docker container, use the `docker cp` command. The basic syntax is:

```
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```


In [17]:
print(f"""
LLM input costs for 1000 requests: ${len(tokens) * 0.005},
LLM output costs for 1000 requests: ${llm_tokens['total_tokens'] * 0.015}
""")



LLM input costs for 1000 requests: $1.605,
LLM output costs for 1000 requests: $5.595

